In [1]:
import csv
import nltk
import pandas as pd

from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [3]:
# Load dataset
dataset = pd.read_csv('dataset_acordaos.csv', delimiter=',')

In [10]:
set(dataset['codigos_movimentos_temas'])

{85568, 85696, 85714, 85721, 85738}

In [15]:
dataset['formatado'][0]

' Registro: 2020.0000079227 ACÓRDÃO Vistos, relatados e discutidos estes autos de Apelação Cível nº 0000150-84.2015.8.26.0315, da Comarca de Laranjal Paulista, em que é apelante BANCO DO BRASIL S/A, é apelado ANA CRISTIANE CARNIEL BRONZATTI. ACORDAM, em sessão permanente e virtual da 17ª Câmara de Direito Privado do Tribunal de Justiça de São Paulo, proferir a seguinte decisão: Deram provimento em parte ao recurso. V. U., de conformidade com o voto do relator, que integra este acórdão. O julgamento teve a participação dos Desembargadores IRINEU FAVA (Presidente) e SOUZA LOPES. São Paulo, 10 de fevereiro de 2020. JOÃO BATISTA VILHENA Relator Assinatura Eletrônica APELANTE: BANCO DO BRASIL S/A APELADO: ANA CRISTIANE CARNIEL BRONZATTI VOTO nº97.406 EMENTA APELAÇÃO EXPURGOS INFLACIONÁRIOS AÇÃO CIVIL PÚBLICA EXECUÇÃO INDIVIDUAL Fase de liquidação de sentença Necessidade de observância do disposto no art. 509, inc. II, do CPC de 2015 Caso concreto em que efetivamente não houve qualquer preju

In [7]:
len(set(dataset['numero_processo']))

1337

In [35]:
dataset_pl = dataset.loc[dataset['codigos_movimentos_temas'] == 85738]
sentences = list(dataset_pl['formatado'])

In [36]:
len(sentences)

534

# Creating a vocabulary

In [21]:
# NLTK portuguese stopwords set
stop_words = set(stopwords.words('portuguese'))
stop_words.remove("não")

In [22]:
words = set()

for sentence in tqdm(sentences):
    if isinstance(sentence, str):
        cur_words = [word.lower() 
                     for word in 
                     nltk.word_tokenize(sentence) if 
                     (word.isalnum() and word.lower() not in stop_words)]
    words.update(cur_words)

100%|██████████| 579/579 [00:22<00:00, 26.09it/s]


In [23]:
with open('vocabulary.txt', 'w') as vocabulary:
    for word in words:
        vocabulary.write(word)
        vocabulary.write('\n')

# Creating a .tsv

In [24]:
ren_sentences = []

for sentence in tqdm(sentences):
    if (isinstance(sentence, str)):
        cur_words = [word.lower() 
                     for word in 
                     nltk.word_tokenize(sentence) if 
                     (word.isalnum() and word.lower() not in stop_words)]
        
        new_sentence = ' '.join(cur_words)
        if (new_sentence.strip() != ""):
            ren_sentences.append(new_sentence)

100%|██████████| 579/579 [00:22<00:00, 26.15it/s]


In [25]:
# Split train, test and validation
X_train, X_rem = train_test_split(ren_sentences, train_size=0.7)
X_test, X_val = train_test_split(X_rem, test_size=0.5)

In [26]:
# Add labels
train_list = [[sentence, 'train'] for sentence in X_train]
test_list = [[sentence, 'test'] for sentence in X_test]
val_list = [[sentence, 'val'] for sentence in X_val]

# Concat lists
concat_list = train_list + test_list + val_list

In [27]:
# Create Dataframe
corpus_df = pd.DataFrame(concat_list, columns=['text', 'train']).replace('"','', regex=True)
corpus_df.to_csv('corpus.tsv', sep='\t', index=False, header=False)
corpus_df

,text,train
0,registro acórdão vistos relatados discutidos a...,train
1,registro acórdão vistos relatados discutidos a...,train
2,registro acórdão vistos relatados discutidos a...,train
3,registro acórdão vistos relatados discutidos a...,train
4,registro acórdão vistos relatados discutidos a...,train
...,...,...
574,registro acórdão vistos relatados discutidos a...,val
575,registro acórdão vistos relatados discutidos a...,val
576,registro acórdão vistos relatados discutidos a...,val
577,registro acórdão vistos relatados discutidos a...,val
